<a href="https://colab.research.google.com/github/satojkovic/hf_vlm/blob/main/hf_vlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", token=userdata.get('HF_TOKEN'))
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    token=userdata.get('HF_TOKEN')
)
model.to(device)


In [ ]:
from PIL import Image
import requests

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"

inputs = processor(image, prompt, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=100)

In [ ]:
print(processor.decode(output[0], skip_special_tokens=True))

# SFT using TRL

The full example script can be found in sft_vlm.py

rename examples/scripts/{vsft_llava.py => sft_vlm.py}

In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
!pip install -U trl

In [ ]:
import torch

if not torch.cuda.is_available():
    raise SystemError("GPU not available. Please change runtime type to include a GPU.")
else:
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")


In [ ]:
# @title 3. Import required libraries
from datasets import load_dataset
from transformers import AutoModelForVision2Seq, AutoProcessor, LlavaForConditionalGeneration
from trl import (
    ModelConfig,
    SFTConfig,
    SFTTrainer,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)
import os

In [ ]:
# @title 4. Setting parameters
# --------------------------------------------------------------------------------
# Script Arguments
# --------------------------------------------------------------------------------
dataset_name: str = "HuggingFaceH4/llava-instruct-mix-vsft" # 元のデータセット
# dataset_name: str = "ydshieh/llava-chat-hf-subsample-blip-caption" # より小さいサブサンプルデータセット (テスト用)
# dataset_name: str = "liuhaotian/LLaVA-Pretrain" # LLaVA論文のデータセットの一つ（形式確認が必要）

dataset_train_split: str = "train"
dataset_test_split: str = "test" # testスプリットがないデータセットの場合、eval_strategy="no" にするか、trainスプリットを分割する必要がある


In [ ]:
# --------------------------------------------------------------------------------
# Model Arguments
# --------------------------------------------------------------------------------
# model_name_or_path: str = "llava-hf/llava-1.5-7b-hf" # 元のモデル
model_name_or_path: str = "llava-hf/llava-1.5-7b-hf" # 小さいモデルでテストしたい場合は変更
# model_name_or_path: str = "llava-hf/llava-v1.6-mistral-7b-hf" # Transformers >= 4.45
# model_name_or_path: str = "meta-llama/Llama-3.2-11B-Vision-Instruct" # Transformers >= 4.45.1 (Colab無料枠ではメモリ不足の可能性大)

model_revision: str = None # "main" or specific commit hash
attn_implementation: str = None # "flash_attention_2" or None. Noneで自動選択。T4ではFlashAttention2は非対応の場合が多い
torch_dtype_str: str = "bfloat16" # "bfloat16", "float16", "auto". T4はbfloat16をサポート
trust_remote_code: bool = True # LLaVAモデルではTrueが必要な場合が多い

# PEFT (LoRA) Configuration - Colabでのメモリ削減のためLoRAを有効化
use_peft: bool = True
peft_lora_r: int = 16 # LoRA rank
peft_lora_alpha: int = 32 # LoRA alpha
# peft_target_modules: list = None # 自動検出に任せるか、モデルに合わせて指定 ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
# LLaVA-1.5 (Llamaベース) の一般的なターゲットモジュール
peft_target_modules: list = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# Quantization Configuration (QLoRAを使う場合)
# load_in_4bit: bool = True # QLoRA (4-bit quantization) を使う場合はTrue
# bnb_4bit_quant_type: str = "nf4" # "nf4" or "fp4"
# bnb_4bit_compute_dtype_str: str = "bfloat16" # "bfloat16", "float16"
load_in_4bit: bool = False # まずはLoRA + bf16で試す
bnb_4bit_quant_type: str = "nf4"
bnb_4bit_compute_dtype_str: str = "bfloat16"

In [ ]:
# --------------------------------------------------------------------------------
# SFT Training Arguments (Hugging Face TrainingArguments)
# --------------------------------------------------------------------------------
output_dir: str = "sft-llava-colab-test"
# per_device_train_batch_size: int = 8 # 元の設定
# gradient_accumulation_steps: int = 8 # 元の設定
per_device_train_batch_size: int = 1 # Colab T4 (16GB VRAM) のメモリ制約のため削減
gradient_accumulation_steps: int = 16 # 実効バッチサイズを 1 * 16 = 16 に維持 (元の 8*8=64 よりは小さい)

# num_train_epochs: float = 3.0 # 元のデフォルト
num_train_epochs: float = 1.0 # テストのため短縮
learning_rate: float = 1e-4 # LoRAの場合、少し高めの学習率が有効なことがある
lr_scheduler_type: str = "cosine"
optim: str = "paged_adamw_8bit" if load_in_4bit or use_peft else "adamw_torch" # QLoRA/LoRAならpaged_adamw_8bit

logging_steps: int = 10
# eval_strategy: str = "steps" # testスプリットがある場合
eval_strategy: str = "no" # 小さいテストデータセットにはtestスプリットがない場合があるため "no" に。あれば "steps"
# eval_steps: int = 100 # eval_strategy="steps" の場合
save_strategy: str = "steps"
save_steps: int = 200 # 進行状況を保存する頻度
# save_total_limit: int = 1 # 古いチェックポイントを削除する場合

bf16: bool = (torch_dtype_str == "bfloat16" and not load_in_4bit) # QLoRAの場合、bf16はcompute_dtypeで指定
fp16: bool = (torch_dtype_str == "float16" and not load_in_4bit) # QLoRAの場合、fp16はcompute_dtypeで指定

gradient_checkpointing: bool = True
# gradient_checkpointing_kwargs = dict(use_reentrant=False) # スクリプト通り
# remove_unused_columns = False # スクリプト通り
# dataset_kwargs = {"skip_prepare_dataset": True} # スクリプト通り

report_to: str = "none" # "tensorboard", "wandb", "none"
push_to_hub: bool = False
# hub_model_id: str = "your-username/sft-llava-colab" # push_to_hub=True の場合に設定


In [ ]:
# --------------------------------------------------------------------------------
# Instantiate Config Objects
# --------------------------------------------------------------------------------
model_args = ModelConfig(
    model_name_or_path=model_name_or_path,
    model_revision=model_revision,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype_str if not load_in_4bit else None, # QLoRAの場合はNoneにし、quantization_configで設定
    trust_remote_code=trust_remote_code,
    use_peft=use_peft,
    lora_r=peft_lora_r,
    lora_alpha=peft_lora_alpha,
    # peft_target_modules=peft_target_modules, # 指定する場合
    lora_target_modules=peft_target_modules,
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
)

In [ ]:
training_args = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    optim=optim,
    logging_steps=logging_steps,
    # eval_steps=eval_steps if eval_strategy == "steps" else None,
    save_strategy=save_strategy,
    save_steps=save_steps,
    # save_total_limit=save_total_limit,
    bf16=bf16,
    fp16=fp16,
    gradient_checkpointing=gradient_checkpointing,
    gradient_checkpointing_kwargs=dict(use_reentrant=False), # 元のスクリプトから
    remove_unused_columns=False, # 元のスクリプトから
    dataset_kwargs={"skip_prepare_dataset": True}, # 元のスクリプトから
    report_to=report_to,
    push_to_hub=push_to_hub,
    # hub_model_id=hub_model_id if push_to_hub else None,
    # max_seq_length=2048, # 必要に応じて設定 (SFTTrainerのデフォルトは1024)
)

In [ ]:
# @title 5. Main process (Load model / Data preparation / Training)

################
# Model, Tokenizer & Processor
################
torch_dtype = (
    model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
)
quantization_config = get_quantization_config(model_args) # model_argsから4-bit/8-bit設定を読み込む


In [ ]:
# Determine device_map
if quantization_config is not None:
    # For quantized models (4-bit/8-bit), get_kbit_device_map handles device placement
    device_map_value = get_kbit_device_map()
    print(f"Quantization is enabled. Using device_map from get_kbit_device_map(): {device_map_value}")
elif torch.cuda.is_available():
    # If not quantized and GPU is available, set to "auto"
    # "auto" lets accelerate handle device placement (typically to cuda:0 on single GPU)
    device_map_value = "auto"
    print(f"GPU is available. Setting device_map='{device_map_value}' for non-quantized model.")
else:
    # No quantization and no GPU, so device_map will be None (CPU)
    device_map_value = None
    print("GPU not available or quantization not used. Model will be loaded on CPU if device_map is None.")

In [ ]:
model_kwargs = dict(
    revision=model_args.model_revision,
    attn_implementation=model_args.attn_implementation,
    torch_dtype=torch_dtype,
    device_map=device_map_value, # Use the determined device_map_value
    quantization_config=quantization_config,
)
# QLoRAの場合、torch_dtypeはquantization_configで指定されるため、明示的に渡すとエラーになることがあるので削除
if quantization_config is not None:
    if 'torch_dtype' in model_kwargs:
        del model_kwargs['torch_dtype'] # get_quantization_configが内部でdtypeを扱うため

In [ ]:
print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code
)

In [ ]:
print("Loading model...")
model = AutoModelForVision2Seq.from_pretrained(
    model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, **model_kwargs
)

In [ ]:
# モデルがロードされたデバイスを確認
if hasattr(model, 'device'):
    print(f"Model loaded on device: {model.device}")
else: # PeftModelの場合
    for n, p in model.named_parameters():
        print(f"Parameter {n} on device: {p.device}")
        break

In [ ]:
################
# Create a data collator to encode text and image pairs
################
def collate_fn(examples):
    texts = []
    images_batch = [] # Changed variable name to avoid conflict
    for example in examples:
        # messagesが文字列の場合とリストの場合に対応
        if isinstance(example["messages"], str): # 一部のデータセットは単一文字列でテキストを持つ場合がある
            # この場合、チャットテンプレートを適用するなら、適切な形式に変換する必要がある
            # 例: [{"role": "user", "content": example["messages"]}]
            # ここでは、データセットが期待する形式であることを前提とする
            # もしHuggingFaceH4/llava-instruct-mix-vsft形式なら、messagesはリストのはず
            # テスト用データセット `ydshieh/llava-chat-hf-subsample-blip-caption` は 'text' フィールドを持つ
            if "text" in example and "images" in example: # For ydshieh's dataset
                 # ydshieh/llava-chat-hf-subsample-blip-caption's 'text' field is like "USER: <image>\nWhat is this? ASSISTANT: This is a cat."
                 # We need to convert this to messages format.
                 # This is a simplified conversion, actual conversion might need more robust parsing.
                parts = example["text"].split("ASSISTANT:")
                user_content = parts[0].replace("USER:", "").strip()
                assistant_content = parts[1].strip() if len(parts) > 1 else ""
                # <image> トークンはprocessor.apply_chat_templateが処理することを期待
                # user_content = user_content.replace("<image>\n", processor.image_token + "\n")
                # LLaVAは <image> をメッセージの先頭に置くことが多い
                # ユーザーの入力に <image> が含まれていることを想定
                example_messages = [{"role": "user", "content": user_content}]
                if assistant_content:
                    example_messages.append({"role": "assistant", "content": assistant_content})
            else: # Assuming example["messages"] is the correct field
                example_messages = example["messages"]
        else:
            example_messages = example["messages"]

        processed_text = processor.apply_chat_template(example_messages, tokenize=False)
        texts.append(processed_text)
        images_batch.append(example["images"]) # imagesはリストのリストになる

    # LLaVA 1.5 (LlavaForConditionalGeneration) は複数画像をサポートしないため、各サンプルから最初の画像のみを使用
    # LLaVA NeXT (LlavaNextForConditionalGeneration) などは複数画像をサポートする場合がある
    # その場合はこの処理をモデルタイプに応じて変更する必要がある
    # AutoModelForVision2Seqでロードした場合、具体的なモデルクラスで判定
    if isinstance(model, LlavaForConditionalGeneration):
        final_images = [img_list[0] for img_list in images_batch if img_list] # 各サンプルの画像リストから最初の画像を取得
    else:
        # 他のモデル (例: LLaVA-NeXT) が複数画像を扱える場合、そのまま渡すか、モデルの期待する形式に合わせる
        # ここでは、データセットが各サンプルに1枚の画像リストを持つと仮定し、それを展開
        final_images = [img_list[0] for img_list in images_batch if img_list] # シンプルに最初の画像を使う

    try:
        batch = processor(text=texts, images=final_images, return_tensors="pt", padding=True)
    except Exception as e:
        print("Error during processor call. Details:")
        print(f"Texts: {texts}")
        print(f"Number of final images: {len(final_images)}")
        # print(f"Final images: {final_images}") # PIL Imageオブジェクトなので表示は省略
        raise e

    labels = batch["input_ids"].clone()
    # Pad tokenのマスク
    if processor.tokenizer.pad_token_id is not None:
        labels[labels == processor.tokenizer.pad_token_id] = -100

    # Image tokenのマスク (モデル特有の処理)
    # LlavaProcessorには image_token があるが、他のプロセッサでは異なる場合がある
    if hasattr(processor, 'image_token'):
        image_token_id = processor.tokenizer.convert_tokens_to_ids(processor.image_token)
        labels[labels == image_token_id] = -100
    elif hasattr(processor.tokenizer, 'additional_special_tokens_ids'):
        # Llama-3.2-Vision-Instructの場合、<image>はspecial tokenとして扱われる
        # Idefics2プロセッサの場合、image_token_id は通常 -100 で、
        # <image> はテキスト中に複数回出現しうる。
        # <fake_token_around_image>のような特殊トークンもマスク対象になることがある。
        # processor.tokenizer.special_tokens_map から '<image>' に対応するIDを取得する。
        # もしくは、モデルのconfigから image_token_index を参照する。
        # For Llama-3.2-Vision, it seems '<image>' is one of the special tokens.
        # The processor might handle image tokens differently, often by inserting placeholders that are later replaced by image embeddings.
        # Let's try to find the ID for a generic image placeholder if not `processor.image_token`
        img_placeholder_id = None
        if "<image>" in processor.tokenizer.get_vocab():
             img_placeholder_id = processor.tokenizer.convert_tokens_to_ids("<image>")
        elif "image_token_index" in model.config.to_dict(): # e.g. Idefics
             img_placeholder_id = model.config.image_token_index

        if img_placeholder_id is not None:
            labels[labels == img_placeholder_id] = -100
            print(f"Masked image placeholder token ID: {img_placeholder_id}")

    batch["labels"] = labels
    return batch

In [ ]:
################
# Dataset
################
print(f"Loading dataset: {dataset_name}")
# HuggingFaceH4/llava-instruct-mix-vsft は images フィールドが PIL Imageのリスト
# ydshieh/llava-chat-hf-subsample-blip-caption は images フィールドが PIL Image (リストではない)
# データセットの形式に合わせて前処理を調整
raw_dataset = load_dataset(dataset_name) # name=script_args.dataset_config (H4データセットはconfigなし)


In [ ]:
# ydshieh/llava-chat-hf-subsample-blip-caption の場合、'images'は単一のPIL Image
# collate_fn が images_batch.append(example["images"]) を期待するため、リストにラップする
# また、'messages'フィールドがないので、'text'から変換する処理をcollate_fnに実装
# def preprocess_dataset(example):
#     if "text" in example and "images" in example: # For ydshieh's dataset
#         # 'messages' フィールドは collate_fn で 'text' から生成するのでここでは何もしない
#         # 'images' をリストにする
#         if not isinstance(example["images"], list):
#             example["images"] = [example["images"]]
#     # H4データセットの場合、'messages'と'images' (リスト) が存在するはず
#     return example

# dataset = raw_dataset.map(preprocess_dataset, batched=False)
dataset = raw_dataset

In [ ]:
# データセットの分割を確認
print("Dataset structure:", dataset)
if dataset_train_split not in dataset:
    print(f"Warning: '{dataset_train_split}' split not found in dataset. Available splits: {list(dataset.keys())}")
    # フォールバックとして最初のスプリットを使用するか、エラーにする
    if list(dataset.keys()):
        actual_train_split_name = list(dataset.keys())[0]
        print(f"Using '{actual_train_split_name}' as train split instead.")
        train_dataset = dataset[actual_train_split_name]
    else:
        raise ValueError("No splits found in the loaded dataset.")
else:
    train_dataset = dataset[dataset_train_split]

In [ ]:
eval_dataset = None
if dataset_test_split not in dataset:
    print(f"Warning: '{dataset_test_split}' split not found. Disabling evaluation.")
    training_args.evaluation_strategy = "no"
else:
    eval_dataset = dataset[dataset_test_split]


In [ ]:
eval_dataset

In [ ]:
print(f"Train dataset size: {len(train_dataset)}")
if eval_dataset:
    print(f"Eval dataset size: {len(eval_dataset)}")


In [ ]:

################
# Training
################
print("Initializing SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=get_peft_config(model_args) if model_args.use_peft else None,
    # max_seq_length=training_args.max_seq_length, # SFTConfigで設定されていれば不要
)

In [ ]:
print("Starting training...")
trainer.train()

print("Training finished.")